### Imports

In [ ]:
from vastdb.session import Session
from dotenv import load_dotenv
import os
import pyarrow as pa
import numpy as np

load_dotenv()
vast_db_access = os.getenv("VAST_DB_ACCESS")
vast_db_secret = os.getenv("VAST_DB_SECRET")
endpoint = os.getenv("VAST_DB_ENDPOINT")
sess = Session(access=vast_db_access, secret=vast_db_secret, endpoint=endpoint)

### Update row(s)

#### update/zero multiplicity

In [ ]:
with sess.transaction() as tx:
    table = tx.bucket("colabfit").schema("dev").table("po_wip")
    reader = table.select(
        predicate=(table["dataset_id"] == "DS_otx1qc9f3pm4_0")
        & (table["multiplicity"] > 1),  # or > 0 if zeroing multiplicity
        internal_row_id=True,
    )
    for batch in reader:
        pbatch = batch.to_pandas()
        pbatch["multiplicity"] = np.int32(1)  # or 0, if zeroing multiplicity
        # print(pbatch.loc[:, ["multiplicity", "$row_id"]])
        pa_table = pa.table(pbatch.loc[:, ["multiplicity", "$row_id"]])
        print(pa_table.schema)
        table.update(pa_table, columns=["multiplicity"])

#### Change dataset names

In [ ]:
ds_name = ""
new_ds_name = ""
with sess.transaction() as tx:
    table = tx.bucket("colabfit").schema("dev").table("ds_wip")
    reader = table.select(
        predicate=(table["name"] == ds_name),
        internal_row_id=True,
    )
    for batch in reader:
        pbatch = batch.to_pandas()
        pbatch["name"] = new_ds_name
        print(pbatch.loc[:, ["name", "$row_id"]])
        pa_table = pa.table(pbatch.loc[:, ["name", "$row_id"]])
        print(pa_table.schema)
        table.update(pa_table, columns=["name"])

#### Edit dataset description

In [ ]:
dataset_id = ""

with sess.transaction() as tx:
    table = tx.bucket("colabfit").schema("dev").table("ds_wip")
    reader = table.select(
        predicate=(table["id"] == dataset_id),
        internal_row_id=True,
    )
    for batch in reader:
        pbatch = batch.to_pandas()
        pbatch["description"] = ""
        print(pbatch.loc[:, ["name", "$row_id"]])
        pa_table = pa.table(pbatch.loc[:, ["name", "$row_id"]])
        print(pa_table.schema)
        table.update(pa_table, columns=["name"])

#### Add dataset DOIs

In [ ]:
session = sess

In [ ]:
doi_map = {
    k: v
    for k, v in zip(
        [
            "DS_82ubxqu96yz7_0",
            "DS_0sa5e1klrpzx_0",
            "DS_gtkq8hnj6tdg_0",
            "DS_2dgg8tui3p9x_0",
            "DS_uluw9723f2n4_0",
            "DS_59dh6cfjgbql_0",
            "DS_y315cvty4fgf_0",
            "DS_72dnci6w4f0s_0",
            "DS_0twwqpqjvftx_0",
            "DS_24d0nebgteu4_0",
            "DS_5yp2nd8n5nau_0",
            "DS_vv60rbjjwpm6_0",
            "DS_xnio123pebli_0",
            "DS_lcjsp7ctc1hy_0",
            "DS_k8m3sm6ves4u_0",
            "DS_gxjhn6vdjnxg_0",
            "DS_6pieq95jrqpn_0",
            "DS_51sddg3b1bp1_0",
            "DS_6qqf55wad1mv_0",
            "DS_izdsom1hklzv_0",
            "DS_bzcf331ql8ji_0",
            "DS_5rjlk0wubpsf_0",
            "DS_lcy9bqjhga53_0",
            "DS_pci59t5zafkx_0",
            "DS_gn4qyaj4yn1x_0",
            "DS_0baqiwozmy40_0",
            "DS_elj5dyxwg3mw_0",
            "DS_bkm2tahmkd0o_0",
            "DS_gk9tv5a9498z_0",
            "DS_5bwpr2n9zxz9_0",
            "DS_h8s4lfyits34_0",
            "DS_s05lflv67rsn_0",
            "DS_xsog5nx0do2h_0",
            "DS_x44bl0q0dyxr_0",
            "DS_dc4lwyrm55p4_0",
            "DS_sk8zwvk3qxur_0",
        ],
        [
            "10.60732/27f8a97a",
            "10.60732/cd74ffdf",
            "10.60732/0396d7de",
            "10.60732/4132ee7c",
            "10.60732/8d6afc67",
            "10.60732/b76ce2d6",
            "10.60732/b52743ef",
            "10.60732/9771a0c2",
            "10.60732/ca55415d",
            "10.60732/89997b6f",
            "10.60732/e77ca63e",
            "10.60732/7cec33e0",
            "10.60732/12246d46",
            "10.60732/8baea040",
            "10.60732/b6f9382a",
            "10.60732/efbb7935",
            "10.60732/8781419f",
            "10.60732/0d5818c5",
            "10.60732/1f7cae3c",
            "10.60732/3c2ddc75",
            "10.60732/41666b82",
            "10.60732/4df848c7",
            "10.60732/8f92dba5",
            "10.60732/c04a4e90",
            "10.60732/4647e973",
            "10.60732/d66c9888",
            "10.60732/ad56ac0a",
            "10.60732/ef8a9926",
            "10.60732/e8c8e0eb",
            "10.60732/61569e2c",
            "10.60732/f5b6ea1b",
            "10.60732/e55c4ce1",
            "10.60732/8ff541c9",
            "10.60732/30653c33",
            "10.60732/b1f21e20",
            "10.60732/b9e7eedf",
        ],
    )
}

In [ ]:
for id, doi in doi_map.items():
    with session.transaction() as tx:
        table = tx.bucket("colabfit-prod").schema("prod").table("ds")
        row = table.select(predicate=table["id"] == id, internal_row_id=True)
        row = row.read_all()

    row = row.to_pandas()
    row["doi"] = doi
    update_table = pa.table(row[["id", "doi", "$row_id"]])
    # print(update_table)
    print(id, doi)
    # break
    with session.transaction() as tx:
        table = tx.bucket("colabfit-prod").schema("prod").table("ds")
        table.update(rows=update_table, columns=["doi"])

In [ ]:
# Add a column value to single row
id = "dataset_id"
col = "publication_year"
value = "2024"
with sess.transaction() as tx:
    table = tx.bucket("colabfit").schema("dev").table("ds_wip")
    row = table.select(predicate=table["id"] == id, internal_row_id=True)
    row = row.read_all()
row = row.to_pandas()
row[col] = value
update_table = pa.table(row[["id", col, "$row_id"]])
with sess.transaction() as tx:
    table = tx.bucket("colabfit").schema("dev").table("ds_wip")
    table.update(rows=update_table, columns=[col])

### Delete a row

In [ ]:
# Delete a row
id = "dataset_id"
with sess.transaction() as tx:
    table = tx.bucket("colabfit").schema("dev").table("ds_wip")
    row = table.select(predicate=table["id"] == id, internal_row_id=True)
    table.delete(row.read_next_batch())